In [ ]:
import numpy as np
import wot.tmap

Load transport map model and cell sets

In [ ]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

Convert cell sets indicating whether a cell at time t is in a cell set

In [ ]:
populations = tmap_model.population_from_cell_sets(cell_sets, at_time=18)
# print cell size sizes
for name in populations:
    p = populations[name].p
    print('{}, {}/{}'.format(name, np.count_nonzero(p), len(p)))

In [ ]:

# dataset has cells on rows and cell sets (trajectories) on columns
trajectory_ds = tmap_model.compute_trajectories(populations)